<a href="https://colab.research.google.com/github/aniishap/ds2002/blob/main/Data_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL Data Processor Project

### Import Necessary Libraries

In [10]:
!pip install PyMySQL
!pip install mysql.connector
!pip install sqlalchemy
!pip install mysql-connector-python
!pip install pyMysql
!pip install pymongo





In [20]:
import os
import datetime
import pymongo
import pprint
import pandas as pd
import numpy
import pandas as pd
from sqlalchemy import create_engine


### Declare & Assign Connection Variables to query MySQL using Python


> Add blockquote



In [83]:


mysql_args = {
    "uid" : "euv7bw",
    "pwd" : "euv7bw!",
    "hostname" : "database.ds2002.org",
    "dbname" : "sakila"
}


### Verify that the connection is succesful

In [84]:
import pymysql

conn = pymysql.connect(
    host=host_name,
    user=user_id,
    password=pwd
)

print("Connected successfully")
conn.close()

Connected successfully


### Define Functions for Getting Data From and Setting Data Into Databases (MySQL, phpMyAdmin)


In [85]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()

    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()

    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")

    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')

    connection.close()


### Create & Populate New Dimension Tables: Extracting Data from Sakila Database

SELECT and extract tables chosen for database idea. From the sakila database, I decided to choose these 5 tables: **film, film_category, category, film_actor, and actor**. I decided to choose these 5 ables because they related to each other through primary and foreign keys. The implication of this database is that these tables can keep track and manage various film types and their inventories. Overall, it keeps track of movie rentals and management.

In [86]:
sql_film = "SELECT * FROM sakila.film;"
df_film = get_dataframe(user_id, pwd, host_name,db_name, sql_film)
df_film.head(5)



,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [65]:
sql_filmcategory = "SELECT * FROM sakila.film_category;"
df_filmcategory = get_dataframe(user_id, pwd, host_name,db_name, sql_filmcategory)
df_filmcategory.head(5)

,film_id,category_id,last_update
0,1,6,2006-02-15 05:07:09
1,2,11,2006-02-15 05:07:09
2,3,6,2006-02-15 05:07:09
3,4,11,2006-02-15 05:07:09
4,5,8,2006-02-15 05:07:09


In [66]:
sql_category = "SELECT * FROM sakila.category;"
df_category = get_dataframe(user_id, pwd, host_name,db_name, sql_category)
df_category.head(5)

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [74]:
sql_filmactor = "SELECT * FROM sakila.film_actor;"
df_filmactor = get_dataframe(user_id, pwd, host_name,db_name, sql_filmactor)
df_filmactor.head(5)

,actor_id,film_id,last_update
0,1,1,2006-02-15 05:05:03
1,1,23,2006-02-15 05:05:03
2,1,25,2006-02-15 05:05:03
3,1,106,2006-02-15 05:05:03
4,1,140,2006-02-15 05:05:03


In [68]:
sql_actor = "SELECT * FROM sakila.actor;"
df_actor = get_dataframe(user_id, pwd, host_name,db_name, sql_actor)
df_actor.head(5)

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


### Perform Any Necessary Transformations to the DataFrames

Columns were dropped from the 5 dataframes to ensure that data is concise and relevant to the selected database design of tracking movie rental types and management.

In [76]:
drop_cols = ['last_update', 'original_language_id', 'special_features', 'description']
df_film.drop(drop_cols, axis=1, inplace=True)
df_film.head(2)

,film_id,title,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating
0,1,ACADEMY DINOSAUR,2006,1,6,0.99,86,20.99,PG
1,2,ACE GOLDFINGER,2006,1,3,4.99,48,12.99,G


In [75]:
drop_cols = ['last_update']
df_filmactor.drop(drop_cols, axis=1, inplace=True)
df_filmactor.head(2)

,actor_id,film_id
0,1,1
1,1,23


In [78]:
drop_cols = ['last_update']
df_filmcategory.drop(drop_cols, axis=1, inplace=True)
df_filmcategory.head(2)

,film_id,category_id
0,1,6
1,2,11


In [80]:
drop_cols = ['last_update']
df_actor.drop(drop_cols, axis=1, inplace=True)
df_actor.head(2)

,actor_id,first_name,last_name
0,1,PENELOPE,GUINESS
1,2,NICK,WAHLBERG


In [82]:
drop_cols = ['last_update']
df_category.drop(drop_cols, axis=1, inplace=True)
df_category.head(2)


,category_id,name
0,1,Action
1,2,Animation


## Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables


In [ ]:

dataframe = df_film
table_name = 'dim_film'
primary_key = 'film_id'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)
